--- Day 12: Rain Risk ---

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

*    Action N means to move north by the given value.
*    Action S means to move south by the given value.
*    Action E means to move east by the given value.
*    Action W means to move west by the given value.
*    Action L means to turn left the given number of degrees.
*    Action R means to turn right the given number of degrees.
*    Action F means to move forward by the given value in the direction the ship is currently facing.

The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

```
F10
N3
F7
R90
F11
```

These instructions would be handled as follows:

*    F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
*    N3 would move the ship 3 units north to east 10, north 3.
*    F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
*    R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
*    F11 would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?


In [1]:
#!fsharp
let testInput = """F10
N3
F7
R90
F11"""

let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
type Heading = N | S | W | E
type Command = N | S | W | E | F | L | R
type Instruction = { command: Command; value: int }
type State = { direction: Heading; x: int; y : int }

In [1]:
#!fsharp
let parse (input:string) : Instruction seq =
    input.Replace("\r", "").Split("\n")
    |> Seq.map
        ( fun row -> 
            let c = row.[0]
            let a = int row.[1..]
            match c with 
            | 'N' -> { command = N; value = a }
            | 'S' -> { command = S; value = a }
            | 'W' -> { command = W; value = a }
            | 'E' -> { command = E; value = a }
            | 'F' -> { command = F; value = a }
            | 'L' -> { command = L; value = a }
            | 'R' -> { command = R; value = a }
        )

testInput |> parse

index,command,value
0,F,10
1,N,3
2,F,7
3,R,90
4,F,11


In [1]:
#!fsharp
let turn (state:State) degrees =
    let order = [| Heading.N; Heading.E; Heading.S; Heading.W |]
    let steps = degrees / 90
    let current = Array.IndexOf( order, state.direction )
    { state with direction = order.[ ( current + 4 + steps ) % 4 ] }

let runStep (state: State) (instruction: Instruction) =
    match instruction.command with 
    | N -> { state with y = state.y - instruction.value }
    | S -> { state with y = state.y + instruction.value }
    | W -> { state with x = state.x - instruction.value }
    | E -> { state with x = state.x + instruction.value }            
    | F -> 
        // we might have out-typed ourselves here
        match state.direction with
        | Heading.N -> { state with y = state.y - instruction.value }
        | Heading.S -> { state with y = state.y + instruction.value }
        | Heading.W -> { state with x = state.x - instruction.value }
        | Heading.E -> { state with x = state.x + instruction.value }            
    | L -> turn state -instruction.value
    | R -> turn state instruction.value

runStep { direction = Heading.E; x = 0; y = 0 } { command = R; value = 270 }

direction,x,y
N,0,0


In [1]:
#!fsharp
let runAll instructions = 
    Seq.fold
        runStep
        { direction = Heading.E; x = 0; y = 0 }
        instructions

testInput 
|> parse 
|> runAll

direction,x,y
S,17,8


In [1]:
#!fsharp
let manhattan state =
    Math.Abs(state.x) + Math.Abs(state.y)

testInput |> parse |> runAll |> manhattan

25

In [1]:
#!fsharp
taskInput |> parse |> runAll |> manhattan

923

--- Part Two ---

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

*    Action N means to move the waypoint north by the given value.
*    Action S means to move the waypoint south by the given value.
*    Action E means to move the waypoint east by the given value.
*    Action W means to move the waypoint west by the given value.
*    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
*    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
*    Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

*    F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
*    N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
*    F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
*    R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
*    F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?


In [1]:
#!fsharp
type State2 = { x: int; y : int; wx: int; wy: int}

let turn2 (state: State2) degrees =
    let steps = (degrees + 360 ) % 360
    // skipping sin/cos...
    match steps with
    | 0 -> state
    | 90 -> { state with wy = state.wx; wx = -state.wy }
    | 180 -> { state with wx = -state.wx; wy = -state.wy }
    | 270 -> { state with wy = -state.wx; wx = state.wy }
    | _ -> failwith "Uknown turn"

let runStep2 (state: State2) (instruction: Instruction) =
    match instruction.command with
    | Command.N -> { state with wy = state.wy - instruction.value }
    | Command.S -> { state with wy = state.wy + instruction.value }
    | Command.W -> { state with wx = state.wx - instruction.value }
    | Command.E -> { state with wx = state.wx + instruction.value }
    | Command.F -> { state with x = state.x + instruction.value * state.wx; y = state.y + instruction.value * state .wy }
    | Command.L -> turn2 state -instruction.value
    | Command.R -> turn2 state instruction.value

let runAll2 instructions =
    Seq.fold
        runStep2
        { x = 0; y = 0; wx = 10; wy = -1 }
        instructions

let manhattan2 state =
    Math.Abs(state.x) + Math.Abs(state.y)

testInput |> parse |> Seq.take 5 |> runAll2 |> manhattan2

286

In [1]:
#!fsharp
taskInput |> parse |> runAll2 |> manhattan2

24769